Website scrapped : https://www.fatsecret.co.id/

In [21]:
import httplib2
from bs4 import BeautifulSoup, SoupStrainer
import requests
import re 

In [14]:
def getHTMLdocument(url): 
      
    # request for HTML document of given url 
    response = requests.get(url) 
      
    # response will be provided in JSON format 
    return response.text 


# Scraping Each Categories

In [33]:
categories = {
    "Buah Buahan" : "https://www.fatsecret.co.id/kalori-gizi/grup/buah",
    "Daging" : "https://www.fatsecret.co.id/kalori-gizi/grup/daging",
    "Ikan & Seafood" : "https://www.fatsecret.co.id/kalori-gizi/grup/ikan-seafood",
    "Kacang Kacangan" : "https://www.fatsecret.co.id/kalori-gizi/grup/kacang-kacangan",
    "Susu" : "https://www.fatsecret.co.id/kalori-gizi/grup/keju%C2%A0-susu",
    "Karbohidrat" : "https://www.fatsecret.co.id/kalori-gizi/grup/nasi%C2%A0mie%C2%A0-%C2%A0pasta",
    "Sup" : "https://www.fatsecret.co.id/kalori-gizi/grup/sup",
    "Telur" : "https://www.fatsecret.co.id/kalori-gizi/grup/telur",
    "Sayur" : "https://www.fatsecret.co.id/kalori-gizi/grup/sayuran",
    "Snack (Makanan Ringan)" : "https://www.fatsecret.co.id/kalori-gizi/grup/makanan-ringan"
}

In [207]:

# List to hold the food items
all_foods = []

for index, (key, url) in enumerate(categories.items()):
    print(f"Scraping {key} - {url}")
    
    # Fetch the page content
    response = requests.get(url)
    
    if response.status_code == 200:
        page_content = response.content
        
        # Parse the HTML content with Beautiful Soup
        soup = BeautifulSoup(page_content, 'html.parser')
        
        # Find all sections within the main div
        sec_holder = soup.find('div', class_='secHolder')
        
        if sec_holder:
            # Extract each h2 and corresponding food_links div
            sections = sec_holder.find_all('h2')
            
            for section in sections:
                # Get the main food item link
                main_food_link = section.find('a')
                main_food_name = main_food_link.get_text()
                main_food_url = main_food_link['href']
                
                # Add main food item to the list
                all_foods.append({'foodname': main_food_name, 'link': main_food_url})
                
                # Find the following sibling div with class 'food_links'
                food_links_div = section.find_next_sibling('div', class_='food_links')
                
                if food_links_div:
                    # Find all 'a' tags within this div
                    links = food_links_div.find_all('a')
                    
                    for link in links:
                        sub_food_name = link.get_text()
                        sub_food_url = link['href']
                        all_foods.append({'foodname': sub_food_name, 'link': sub_food_url, 'kategori': key})
                else:
                    print("No additional food links found.")
        else:
            print(f"No 'secHolder' div found for {key}")
    else:
        print(f"Failed to retrieve the page for {key}")

    # Adding a small delay to avoid overloading the server
    import time
    time.sleep(1)

Scraping Buah Buahan - https://www.fatsecret.co.id/kalori-gizi/grup/buah
Scraping Daging - https://www.fatsecret.co.id/kalori-gizi/grup/daging
Scraping Ikan & Seafood - https://www.fatsecret.co.id/kalori-gizi/grup/ikan-seafood
Scraping Kacang Kacangan - https://www.fatsecret.co.id/kalori-gizi/grup/kacang-kacangan
Scraping Susu - https://www.fatsecret.co.id/kalori-gizi/grup/keju%C2%A0-susu
Scraping Karbohidrat - https://www.fatsecret.co.id/kalori-gizi/grup/nasi%C2%A0mie%C2%A0-%C2%A0pasta
Scraping Sup - https://www.fatsecret.co.id/kalori-gizi/grup/sup
Scraping Telur - https://www.fatsecret.co.id/kalori-gizi/grup/telur
Scraping Sayur - https://www.fatsecret.co.id/kalori-gizi/grup/sayuran
Scraping Snack (Makanan Ringan) - https://www.fatsecret.co.id/kalori-gizi/grup/makanan-ringan


# Scraping The Nutrition Value

In [232]:
def scrape_food_details(food_item):
    food_name = food_item['foodname']
    food_cat = food_item['kategori']
    food_link = base_url + food_item['link']
    # Make a GET request to the food link
    response = requests.get(food_link)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        nutrient_elements = soup.find(class_="nutrition_facts")
        # fact_panel = soup.find('div', classs_="nutrition_facts international")
        if nutrient_elements is not None or nutrient_elements:
            # print(nutrient_elements)
            try:
                nut_dict = {}
                nut_dict['Nama'] = food_name
                nut_dict['Kategori'] = food_cat
                nut_dict['Ukuran Porsi'] = nutrient_elements.find(class_='serving_size black serving_size_value').text.strip()
                nut_dict['Kalori'] = nutrient_elements.find(class_='nutrient right tRight').text.strip()
                nut_dict['Lemak'] = nutrient_elements.find_all(class_='nutrient black right tRight')[1].text.strip()
                nut_dict['Lemak Jenuh'] = nutrient_elements.find_all(class_='nutrient right tRight')[1].text.strip()
                nut_dict['Lemak Tak Jenuh Ganda'] = nutrient_elements.find_all(class_='nutrient right tRight')[2].text.strip()
                nut_dict['Lemak Tak Jenuh Tunggal'] = nutrient_elements.find_all(class_='nutrient right tRight')[3].text.strip()
                nut_dict['Kolestrol'] = nutrient_elements.find_all(class_='nutrient right tRight')[4].text.strip()
                nut_dict['Protein'] = nutrient_elements.find_all(class_='nutrient black right tRight')[2].text.strip()
                nut_dict['Karbohidrat'] = nutrient_elements.find_all(class_='nutrient black right tRight')[3].text.strip()
                nut_dict['Serat'] = nutrient_elements.find_all(class_='nutrient right tRight')[5].text.strip()
                nut_dict['Gula'] = nutrient_elements.find_all(class_='nutrient right tRight')[6].text.strip()
                nut_dict['Sodium'] = nutrient_elements.find_all(class_='nutrient right tRight')[7].text.strip()
                nut_dict['Kalium'] = nutrient_elements.find_all(class_='nutrient right tRight')[8].text.strip()
                # for nutrient in nutrient_elements:
                #     nutrient_name = nutrient.get_text(strip=True)                
                #     return nutrient_name
                return nut_dict
            except:
                pass
        else:
            return None
    else:
        print(f"Failed to retrieve the page for {food_name}. Status code: {response.status_code}")

In [233]:
import time

base_url = "https://www.fatsecret.co.id"
processed_all_foods = []
for item in all_foods:
    if 'kategori' in item:
        processed_all_foods.append(scrape_food_details(item))
    else:
        del(item)
    time.sleep(2)
    # break
print(processed_all_foods)

[{'Nama': 'Alpukat', 'Kategori': 'Buah Buahan', 'Ukuran Porsi': '100 gram (g)', 'Kalori': '160 kkal', 'Lemak': '14,66g', 'Lemak Jenuh': '2,126g', 'Lemak Tak Jenuh Ganda': '1,816g', 'Lemak Tak Jenuh Tunggal': '9,799g', 'Kolestrol': '0mg', 'Protein': '2g', 'Karbohidrat': '8,53g', 'Serat': '6,7g', 'Gula': '0,66g', 'Sodium': '7mg', 'Kalium': '485mg'}, {'Nama': 'Guacamole', 'Kategori': 'Buah Buahan', 'Ukuran Porsi': '1 sdm', 'Kalori': '23 kkal', 'Lemak': '2,08g', 'Lemak Jenuh': '0,302g', 'Lemak Tak Jenuh Ganda': '0,258g', 'Lemak Tak Jenuh Tunggal': '1,393g', 'Kolestrol': '0mg', 'Protein': '0,29g', 'Karbohidrat': '1,24g', 'Serat': '1g', 'Gula': '0,1g', 'Sodium': '22mg', 'Kalium': '69mg'}, {'Nama': 'Guacamole dengan Tomat', 'Kategori': 'Buah Buahan', 'Ukuran Porsi': '100 gram (g)', 'Kalori': '116 kkal', 'Lemak': '9,94g', 'Lemak Jenuh': '1,446g', 'Lemak Tak Jenuh Ganda': '1,258g', 'Lemak Tak Jenuh Tunggal': '6,625g', 'Kolestrol': '0mg', 'Protein': '1,62g', 'Karbohidrat': '7,64g', 'Serat': '4,9

In [235]:
new_processed = []
for i in processed_all_foods:
    if i is not None:
        new_processed.append(i)

In [236]:
import pandas as pd

new_df = pd.DataFrame(new_processed)

In [237]:
new_df

,Nama,Kategori,Ukuran Porsi,Kalori,Lemak,Lemak Jenuh,Lemak Tak Jenuh Ganda,Lemak Tak Jenuh Tunggal,Kolestrol,Protein,Karbohidrat,Serat,Gula,Sodium,Kalium
0,Alpukat,Buah Buahan,100 gram (g),160 kkal,"14,66g","2,126g","1,816g","9,799g",0mg,2g,"8,53g","6,7g","0,66g",7mg,485mg
1,Guacamole,Buah Buahan,1 sdm,23 kkal,"2,08g","0,302g","0,258g","1,393g",0mg,"0,29g","1,24g",1g,"0,1g",22mg,69mg
2,Guacamole dengan Tomat,Buah Buahan,100 gram (g),116 kkal,"9,94g","1,446g","1,258g","6,625g",0mg,"1,62g","7,64g","4,9g","1,37g",201mg,393mg
3,Guacamole dengan Tomat dan Cabe Paprika,Buah Buahan,1 mangkok,266 kkal,"22,72g","3,304g","2,882g","15,129g",0mg,"3,77g","17,75g","11,4g","3,31g",459mg,913mg
4,Buah Anggur,Buah Buahan,1 tanpa biji,3 kkal,"0,01g","0,003g","0,002g",0g,0mg,"0,04g","0,9g",0g,"0,77g",0mg,10mg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,Sushi,Snack (Makanan Ringan),1 buah,37 kkal,"0,11g","0,023g","0,033g","0,033g",1mg,"1,12g","7,77g","0,2g","1,63g",155mg,34mg
448,Sushi dengan Sayuran dan Seafood,Snack (Makanan Ringan),1 buah,36 kkal,"0,11g","0,026g","0,032g","0,023g",2mg,"1,4g","7,15g","0,3g","1,47g",56mg,34mg
449,Sushi dengan Sayuran,Snack (Makanan Ringan),1 buah,38 kkal,"0,07g","0,018g","0,023g","0,017g",0mg,"0,8g","8,24g","0,3g","1,7g",62mg,26mg
450,Sushi dengan Sayuran yang Digulung dengan Rump...,Snack (Makanan Ringan),1 buah,31 kkal,"0,06g","0,017g","0,018g","0,018g",0mg,"0,57g","6,71g","0,1g","1,16g",25mg,16mg


In [238]:
new_df.to_csv("dataset/dataset.csv")